In [3]:
#csv 파일 읽어서 sql파일에 넣기

import pymysql
import pandas as pd

def load_db_score_data():
    xlfile = 'C:/Users\yoon2/datascience/db_score_3_labels.xlsx'
    #xlfile = './datascience/db_score_3_labels.xlsx'
    db_score = pd.read_excel(xlfile)
    
    #db connection 생성
    conn = pymysql.connect(host = 'localhost',user = 'root',password = 'taeyoon!', db = 'datascienceassignment4')
    curs = conn.cursor(pymysql.cursors.DictCursor)

    #겹칠 경우
    drop_sql = """drop table if exists db_score"""
    curs.execute(drop_sql)
    conn.commit()
    
    #직접 데이터프레임으로부터 테이블을 만들고 입력 하는 모듈
    import sqlalchemy
    database_username = 'root'
    database_password = 'taeyoon!'
    database_ip = 'localhost'
    database_name = 'datascienceassignment4'
    database_connection = sqlalchemy.create_engine('mysql+pymysql://{0}:{1}@{2}/{3}'.format(database_username,database_password,database_ip,database_name))
    
    #db_score 테이블 생성 + 데이터 입력
    db_score.to_sql(con=database_connection, name='db_score',if_exists='replace')
    
load_db_score_data()

conn = pymysql.connect(host = 'localhost',user = 'root',password = 'taeyoon!', db = 'datascienceassignment4')
curs = conn.cursor(pymysql.cursors.DictCursor)

sql="select * from db_score"
curs.execute(sql)

#fetchone이 메모리요소에서 보면 바람직하나 데이터양이 적어 fetchall사용
data=curs.fetchall()

curs.close()
conn.close()

#딕셔너리 형태가 튜플 형태의 리스트로 바뀐다.
X=[(t['homework'],t['discussion'],t['midterm'])for t in data]

#tuple의 리스트가 numpy로 바뀐다.
import numpy as np
X=np.array(X)

y=[1 if (t['grade']=='B') else -1 for t in data]
y=np.array(y)

#----------------------------SVM--------------------------------------------------
print("SVM--------------------------------------------------------------------")
print("(1) train_test_split\n")
#(1) train_test_split

#train data와 test data분리
from sklearn.model_selection import train_test_split
#test_size=0.33 - train데이터(7)와 test data(3)를 7대3으로 나눈다
#random_state=42 data를 섞는다
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.33, random_state=42) #보통 42로 고정

'''
#특성 스케일 - 특성 간 값의 범위 줄이
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)
'''
#훈련셋으로 SVM 훈련
from sklearn.svm import SVC
svm_model=SVC(kernel='rbf',C=8,gamma=0.1)
svm_model.fit(X_train,y_train)
#테스트셋을 가지고 성능 확인
y_predict = svm_model.predict(X_test) # 테스트
 

#성능 측정
def classification_performance_eval(y,y_predict):
    tp, tn, fp, fn = 0, 0, 0, 0
    for y, yp in zip(y,y_predict) :
            if (y == 1 and yp == 1):
                tp += 1
            elif(y == 1 and yp == -1):
                fn += 1
            elif(y== -1 and yp == 1) : 
                fp += 1
            else:
                tn += 1
    print("tp : ",tp," fn : ",fn," fp : ",fp," tn : ",tn)
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    precision = (tp)/(tp+fp)
    recall = (tp) / (tp+fn)
    if(recall==0):
        print("recall = 0 ,예외처리")
        f1_score=0
    else:
        f1_score = 2*precision*recall / (precision + recall)
    return accuracy,precision,recall,f1_score

acc, prec, rec, f1 = classification_performance_eval(y_test,y_predict)

print("\naccuracy=%f"%acc)
print("precision=%f"%prec)
print("recall=%f"%rec)
print("f1_score=%f\n"%f1)

#(2)K-Flod cross validation
print("(2) K-Flod cross validation\n")
#정확한 성능 구하기
from sklearn.model_selection import KFold
accuracy= []
precision = []
recall = []
f1_score = []
#n_splits값을 너무 작게 train이 충분히 안된다.
kf = KFold(n_splits=4, random_state=42, shuffle=True)

for train_index,test_index in kf.split(X):
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
    # 모델 변경

    #훈련셋으로 SVM 훈련
    svm_model=SVC(kernel='rbf',C=8,gamma=0.1)
    svm_model.fit(X_train,y_train)
    #테스트셋을 가지고 성능 확인
    y_predict = svm_model.predict(X_test) # 테스트
    acc, prec, rec, f1 = classification_performance_eval(y_test,y_predict) 
    accuracy.append(acc)
    precision.append(prec)
    recall.append(rec)
    f1_score.append(f1)

import statistics

print("\naverage_accuracy = ",statistics.mean(accuracy))
print("average_precision = ",statistics.mean(precision))
print("average_recall = ",statistics.mean(recall))
print("average_f1_score = ",statistics.mean(f1_score))

#----------------------------LogisticRegression------------------------------
print("\nLogisticRegression------------------------------------------------------")
print("(1) train_test_split\n")
#(1) train_test_split

#train data와 test data분리
from sklearn.model_selection import train_test_split
#test_size=0.33 - train데이터(7)와 test data(3)를 7대3으로 나눈다
#random_state=42 data를 섞는다
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3, random_state=43) #보통 42로 고정


from sklearn.linear_model import LogisticRegression
ml=LogisticRegression(C=1000.0,random_state=0)
ml.fit(X_train,y_train)
#테스트셋을 가지고 성능 확인
y_predict=ml.predict(X_test)

#성능 측정
def classification_performance_eval(y,y_predict):
    tp, tn, fp, fn = 0, 0, 0, 0
    for y, yp in zip(y,y_predict) :
            if (y == 1 and yp == 1):
                tp += 1
            elif(y == 1 and yp == -1):
                fn += 1
            elif(y== -1 and yp == 1) : 
                fp += 1
            else:
                tn += 1
    
    print("tp : ",tp," fn : ",fn," fp : ",fp," tn : ",tn)
    accuracy = (tp+tn)/(tp+tn+fp+fn)

    if(tp==0):
        print("tp = 0 ,precision 예외처리")
        precision=0
    else:
        precision = (tp)/(tp+fp)
    recall = (tp) / (tp+fn)
    if(recall==0):
        print("recall = 0 ,f1_score 예외처리")
        f1_score=0
    else:
        f1_score = 2*precision*recall / (precision + recall)

    return accuracy,precision,recall,f1_score
acc, prec, rec, f1 = classification_performance_eval(y_test,y_predict)

print("\naccuracy=%f"%acc)
print("precision=%f"%prec)
print("recall=%f"%rec)
print("f1_score=%f\n"%f1)

#(2)K-Flod cross validation
print("(2) K-Flod cross validation\n")
#정확한 성능 구하기
from sklearn.model_selection import KFold
accuracy= []
precision = []
recall = []
f1_score = []
#n_splits값을 너무 작게 train이 충분히 안된다.
kf = KFold(n_splits=2, random_state=500, shuffle=True)
#random_state 42에서 46으로 조정
for train_index,test_index in kf.split(X):
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
    # 모델 변경

    #훈련셋으로 LogisticRegression 훈련
    ml=LogisticRegression(C=1000.0,random_state=0)
    ml.fit(X_train,y_train)
    #테스트셋을 가지고 성능 확인
    y_predict=ml.predict(X_test)
    
    acc, prec, rec, f1 = classification_performance_eval(y_test,y_predict) 
    accuracy.append(acc)
    precision.append(prec)
    recall.append(rec)
    f1_score.append(f1)

import statistics

print("\naverage_accuracy = ",statistics.mean(accuracy))
print("average_precision = ",statistics.mean(precision))
print("average_recall = ",statistics.mean(recall))
print("average_f1_score = ",statistics.mean(f1_score))


#----------------------------Random Forest------------------------------
print("\nRandom Forest-------------------------------------------------------")
print("(1) train_test_split\n")
#(1) train_test_split

#train data와 test data분리
from sklearn.model_selection import train_test_split
#test_size=0.33 - train데이터(7)와 test data(3)를 7대3으로 나눈다
#random_state=42 data를 섞는다
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42) #보통 42로 고정

from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(random_state=0)
rf.fit(X_train,y_train)
#테스트셋을 가지고 성능 확인
y_predict=rf.predict(X_test)

#성능 측정
def classification_performance_eval(y,y_predict):
    tp, tn, fp, fn = 0, 0, 0, 0
    
    for y, yp in zip(y,y_predict) :
            if (y == 1 and yp == 1):
                tp += 1
            elif(y == 1 and yp == -1):
                fn += 1
            elif(y== -1 and yp == 1) : 
                fp += 1
            else:
                tn += 1
    print("tp : ",tp," fn : ",fn," fp : ",fp," tn : ",tn)
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    precision = (tp)/(tp+fp)
    recall = (tp) / (tp+fn)
    f1_score = 2*precision*recall / (precision + recall)
    return accuracy,precision,recall,f1_score
acc, prec, rec, f1 = classification_performance_eval(y_test,y_predict)

print("\naccuracy=%f"%acc)
print("precision=%f"%prec)
print("recall=%f"%rec)
print("f1_score=%f\n"%f1)

#(2)K-Flod cross validation
print("(2) K-Flod cross validation\n")
#정확한 성능 구하기
from sklearn.model_selection import KFold
accuracy= []
precision = []
recall = []
f1_score = []
#n_splits값을 너무 작게 train이 충분히 안된다.
kf = KFold(n_splits=5, random_state=42, shuffle=True)
for train_index,test_index in kf.split(X):
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
    # 모델 변경
    
    #훈련셋으로 RandomForest 훈련
    rf=RandomForestClassifier(random_state=0)
    rf.fit(X_train,y_train)
    #테스트셋을 가지고 성능 확인
    y_predict=rf.predict(X_test)
    
    acc, prec, rec, f1 = classification_performance_eval(y_test,y_predict) 
    accuracy.append(acc)
    precision.append(prec)
    recall.append(rec)
    f1_score.append(f1)

import statistics
print("\naverage_accuracy = ",statistics.mean(accuracy))
print("average_precision = ",statistics.mean(precision))
print("average_recall = ",statistics.mean(recall))
print("average_f1_score = ",statistics.mean(f1_score))


SVM--------------------------------------------------------------------
(1) train_test_split

tp :  2  fn :  7  fp :  4  tn :  18

accuracy=0.645161
precision=0.333333
recall=0.222222
f1_score=0.266667

(2) K-Flod cross validation

tp :  3  fn :  3  fp :  4  tn :  13
tp :  4  fn :  4  fp :  6  tn :  9
tp :  2  fn :  7  fp :  2  tn :  12
tp :  5  fn :  3  fp :  6  tn :  9

average_accuracy =  0.6195652173913043
average_precision =  0.4457792207792208
average_recall =  0.4618055555555556
average_f1_score =  0.43499775078722447

LogisticRegression------------------------------------------------------
(1) train_test_split

tp :  2  fn :  4  fp :  8  tn :  14

accuracy=0.571429
precision=0.200000
recall=0.333333
f1_score=0.250000

(2) K-Flod cross validation

tp :  2  fn :  14  fp :  6  tn :  24
tp :  2  fn :  13  fp :  7  tn :  24

average_accuracy =  0.5652173913043478
average_precision =  0.2361111111111111
average_recall =  0.12916666666666665
average_f1_score =  0.16666666666666669

Ra

In [4]:
#csv 파일 읽어서 sql파일에 넣기

import pymysql
import pandas as pd

def load_db_score_data():
    xlfile = 'C:/Users\yoon2/datascience/db_score_3_labels.xlsx'
    #xlfile = './datascience/db_score_3_labels.xlsx'
    db_score = pd.read_excel(xlfile)
    
    #db connection 생성
    conn = pymysql.connect(host = 'localhost',user = 'root',password = 'taeyoon!', db = 'datascienceassignment4')
    curs = conn.cursor(pymysql.cursors.DictCursor)

    #겹칠 경우
    drop_sql = """drop table if exists db_score"""
    curs.execute(drop_sql)
    conn.commit()
    
    #직접 데이터프레임으로부터 테이블을 만들고 입력 하는 모듈
    import sqlalchemy
    database_username = 'root'
    database_password = 'taeyoon!'
    database_ip = 'localhost'
    database_name = 'datascienceassignment4'
    database_connection = sqlalchemy.create_engine('mysql+pymysql://{0}:{1}@{2}/{3}'.format(database_username,database_password,database_ip,database_name))
    
    #db_score 테이블 생성 + 데이터 입력
    db_score.to_sql(con=database_connection, name='db_score',if_exists='replace')
    
load_db_score_data()

conn = pymysql.connect(host = 'localhost',user = 'root',password = 'taeyoon!', db = 'datascienceassignment4')
curs = conn.cursor(pymysql.cursors.DictCursor)

sql="select * from db_score"
curs.execute(sql)

#fetchone이 메모리요소에서 보면 바람직하나 데이터양이 적어 fetchall사용
data=curs.fetchall()

curs.close()
conn.close()

#딕셔너리 형태가 튜플 형태의 리스트로 바뀐다.
X=[(t['homework'],t['discussion'],t['midterm'])for t in data]

#tuple의 리스트가 numpy로 바뀐다.
import numpy as np
X=np.array(X)

y=[(t['grade']) for t in data]
y=np.array(y)

#----------------------------SVM--------------------------------------------------
print("SVM--------------------------------------------------------------------")
print("(1) train_test_split\n")
#(1) train_test_split

#train data와 test data분리
from sklearn.model_selection import train_test_split
#test_size=0.33 - train데이터(7)와 test data(3)를 7대3으로 나눈다
#random_state=42 data를 섞는다
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.25, random_state=42) #보통 42로 고정

'''
#특성 스케일 - 특성 간 값의 범위 줄이
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)
'''
#훈련셋으로 SVM 훈련
from sklearn.svm import SVC
svm_model=SVC(kernel='rbf',C=8,gamma=0.1)
svm_model.fit(X_train,y_train)
#테스트셋을 가지고 성능 확인
y_predict = svm_model.predict(X_test) # 테스트
 

#성능 측정
def classification_performance_eval(y,y_predict):
    
    #A
    tp, tn, fp, fn = 0, 0, 0, 0

    for y, yp in zip(y_test,y_predict) :
        if (y == 'A' and yp == 'A'):
            tp += 1
        elif((y == 'A' and yp == 'B')or(y == 'A' and yp == 'C')):
            fn += 1 
        elif((y == 'B' and yp == 'B')or(y == 'C' and yp == 'C')) : 
            tn += 1
        else:
            fp += 1      
    #print("tp : ",tp," fn : ",fn," fp : ",fp," tn : ",tn)
    precision_a = (tp)/(tp+fp)
    recall_a = (tp) / (tp+fn)
    f1_score_a = 2*precision_a*recall_a / (precision_a + recall_a)

    #B

    tp, tn, fp, fn = 0, 0, 0, 0

    for y, yp in zip(y_test,y_predict) :
        if (y == 'B' and yp == 'B'):
            tp += 1
        elif((y == 'B' and yp == 'A')or(y == 'B' and yp == 'C')):
            fn += 1
        elif((y == 'A' and yp == 'A')or(y == 'C' and yp == 'C')) : 
            tn += 1
        else:
            fp += 1         

    #print("tp : ",tp," fn : ",fn," fp : ",fp," tn : ",tn)
    precision_b = (tp)/(tp+fp)
    recall_b = (tp) / (tp+fn)
    f1_score_b = 2*precision_b*recall_b / (precision_b + recall_b)

    #C
    tp, tn, fp, fn = 0, 0, 0, 0

    for y, yp in zip(y_test,y_predict) :
        if (y == 'C' and yp == 'C'):
            tp += 1
        elif((y == 'C' and yp == 'A')or(y == 'C' and yp == 'B')):
            fn += 1
        elif((y == 'A' and yp == 'A')or(y == 'B' and yp == 'B')) : 
            tn += 1
        else:
            fp += 1
    #print("tp : ",tp," fn : ",fn," fp : ",fp," tn : ",tn)
    precision_c = (tp)/(tp+fp)
    recall_c = (tp) / (tp+fn)
    f1_score_c = 2*precision_c*recall_c / (precision_c + recall_c)

    accuracy = (tp+tn)/(tp+tn+fp+fn)
    
    return accuracy,precision_a,precision_b,precision_c,recall_a,recall_b,recall_c,f1_score_a,f1_score_b,f1_score_c

acc, prec_a, prec_b, prec_c, rec_a, rec_b, rec_c, f1_a, f1_b, f1_c = classification_performance_eval(y_test,y_predict)

print("\naccuracy=%f"%acc)

print("A")
print("precision=%f"%prec_a)
print("recall=%f"%rec_a)
print("f1_score=%f\n"%f1_a)

print("B")
print("precision=%f"%prec_b)
print("recall=%f"%rec_b)
print("f1_score=%f\n"%f1_b)

print("C")
print("precision=%f"%prec_c)
print("recall=%f"%rec_c)
print("f1_score=%f\n"%f1_c)

#(2)K-Flod cross validation
print("(2) K-Flod cross validation\n")
#정확한 성능 구하기
from sklearn.model_selection import KFold
accuracy= []
precision_a = []
precision_b = []
precision_c = []
recall_a = []
recall_b = []
recall_c = []
f1_score_a = []
f1_score_b = []
f1_score_c = []
#n_splits값을 너무 작게 train이 충분히 안된다.
kf = KFold(n_splits=4, random_state=42, shuffle=True)

for train_index,test_index in kf.split(X):
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
    # 모델 변경

    #훈련셋으로 SVM 훈련
    svm_model=SVC(kernel='rbf',C=8,gamma=0.1)
    svm_model.fit(X_train,y_train)
    #테스트셋을 가지고 성능 확인
    y_predict = svm_model.predict(X_test) # 테스트
    acc, prec_a, prec_b, prec_c, rec_a, rec_b, rec_c, f1_a, f1_b, f1_c= classification_performance_eval(y_test,y_predict) 
    accuracy.append(acc)
    precision_a.append(prec_a)
    precision_b.append(prec_b)
    precision_c.append(prec_c)
    recall_a.append(rec_a)
    recall_b.append(rec_b)
    recall_c.append(rec_c)
    f1_score_a.append(f1_a)
    f1_score_b.append(f1_b)
    f1_score_c.append(f1_c)

import statistics
print("\naverage_accuracy = ",statistics.mean(accuracy))
print("A")
print("average_precision = ",statistics.mean(precision_a))
print("average_recall = ",statistics.mean(recall_a))
print("average_f1_score = ",statistics.mean(f1_score_a))
print("B")
print("average_precision = ",statistics.mean(precision_b))
print("average_recall = ",statistics.mean(recall_b))
print("average_f1_score = ",statistics.mean(f1_score_b))
print("C")
print("average_precision = ",statistics.mean(precision_c))
print("average_recall = ",statistics.mean(recall_c))
print("average_f1_score = ",statistics.mean(f1_score_c))

#----------------------------LogisticRegression------------------------------
print("\nLogisticRegression------------------------------------------------------")
print("(1) train_test_split\n")
#(1) train_test_split

#train data와 test data분리
from sklearn.model_selection import train_test_split
#test_size=0.33 - train데이터(7)와 test data(3)를 7대3으로 나눈다
#random_state=42 data를 섞는다
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.25, random_state=43) #보통 42로 고정


from sklearn.linear_model import LogisticRegression
ml=LogisticRegression(C=1000.0,random_state=0,max_iter=10000)
ml.fit(X_train,y_train)
#테스트셋을 가지고 성능 확인
y_predict=ml.predict(X_test)

#성능 측정
def classification_performance_eval(y,y_predict):
    
    #A
    tp, tn, fp, fn = 0, 0, 0, 0

    for y, yp in zip(y_test,y_predict) :
        if (y == 'A' and yp == 'A'):
            tp += 1
        elif((y == 'A' and yp == 'B')or(y == 'A' and yp == 'C')):
            fn += 1 
        elif((y == 'B' and yp == 'B')or(y == 'C' and yp == 'C')) : 
            tn += 1
        else:
            fp += 1      
    #print("tp : ",tp," fn : ",fn," fp : ",fp," tn : ",tn)
    precision_a = (tp)/(tp+fp)
    recall_a = (tp) / (tp+fn)
    f1_score_a = 2*precision_a*recall_a / (precision_a + recall_a)

    #B

    tp, tn, fp, fn = 0, 0, 0, 0

    for y, yp in zip(y_test,y_predict) :
        if (y == 'B' and yp == 'B'):
            tp += 1
        elif((y == 'B' and yp == 'A')or(y == 'B' and yp == 'C')):
            fn += 1
        elif((y == 'A' and yp == 'A')or(y == 'C' and yp == 'C')) : 
            tn += 1
        else:
            fp += 1         

    #print("tp : ",tp," fn : ",fn," fp : ",fp," tn : ",tn)
    precision_b = (tp)/(tp+fp)
    recall_b = (tp) / (tp+fn)
    f1_score_b = 2*precision_b*recall_b / (precision_b + recall_b)

    #C
    tp, tn, fp, fn = 0, 0, 0, 0

    for y, yp in zip(y_test,y_predict) :
        if (y == 'C' and yp == 'C'):
            tp += 1
        elif((y == 'C' and yp == 'A')or(y == 'C' and yp == 'B')):
            fn += 1
        elif((y == 'A' and yp == 'A')or(y == 'B' and yp == 'B')) : 
            tn += 1
        else:
            fp += 1
    #print("tp : ",tp," fn : ",fn," fp : ",fp," tn : ",tn)
    precision_c = (tp)/(tp+fp)
    recall_c = (tp) / (tp+fn)
    f1_score_c = 2*precision_c*recall_c / (precision_c + recall_c)

    accuracy = (tp+tn)/(tp+tn+fp+fn)
    
    return accuracy,precision_a,precision_b,precision_c,recall_a,recall_b,recall_c,f1_score_a,f1_score_b,f1_score_c

acc, prec_a, prec_b, prec_c, rec_a, rec_b, rec_c, f1_a, f1_b, f1_c = classification_performance_eval(y_test,y_predict)

print("\naccuracy=%f"%acc)

print("A")
print("precision=%f"%prec_a)
print("recall=%f"%rec_a)
print("f1_score=%f\n"%f1_a)

print("B")
print("precision=%f"%prec_b)
print("recall=%f"%rec_b)
print("f1_score=%f\n"%f1_b)

print("C")
print("precision=%f"%prec_c)
print("recall=%f"%rec_c)
print("f1_score=%f\n"%f1_c)


#(2)K-Flod cross validation
print("(2) K-Flod cross validation\n")
#정확한 성능 구하기
from sklearn.model_selection import KFold
accuracy= []
precision_a = []
precision_b = []
precision_c = []
recall_a = []
recall_b = []
recall_c = []
f1_score_a = []
f1_score_b = []
f1_score_c = []
#n_splits값을 너무 작게 train이 충분히 안된다.
kf = KFold(n_splits=3, random_state=500, shuffle=True)
#random_state 42에서 46으로 조정
for train_index,test_index in kf.split(X):
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
    # 모델 변경
    #특성 스케일 - 특성 간 값의 범위 줄이
    from sklearn.preprocessing import StandardScaler
    sc=StandardScaler()
    sc.fit(X_train)
    X_train_std = sc.transform(X_train)
    X_test_std = sc.transform(X_test)
    
    #훈련셋으로 LogisticRegression 훈련
    ml=LogisticRegression(C=1000,solver='lbfgs',random_state=42)
    ml.fit(X_train_std,y_train)
    #테스트셋을 가지고 성능 확인
    y_predict=ml.predict(X_test_std)
    
    acc, prec_a, prec_b, prec_c, rec_a, rec_b, rec_c, f1_a, f1_b, f1_c = classification_performance_eval(y_test,y_predict) 
    accuracy.append(acc)
    precision_a.append(prec_a)
    precision_b.append(prec_b)
    precision_c.append(prec_c)
    recall_a.append(rec_a)
    recall_b.append(rec_b)
    recall_c.append(rec_c)
    f1_score_a.append(f1_a)
    f1_score_b.append(f1_b)
    f1_score_c.append(f1_c)

import statistics
print("\naverage_accuracy = ",statistics.mean(accuracy))
print("A")
print("average_precision = ",statistics.mean(precision_a))
print("average_recall = ",statistics.mean(recall_a))
print("average_f1_score = ",statistics.mean(f1_score_a))
print("B")
print("average_precision = ",statistics.mean(precision_b))
print("average_recall = ",statistics.mean(recall_b))
print("average_f1_score = ",statistics.mean(f1_score_b))
print("C")
print("average_precision = ",statistics.mean(precision_c))
print("average_recall = ",statistics.mean(recall_c))
print("average_f1_score = ",statistics.mean(f1_score_c))


#----------------------------Random Forest------------------------------
print("\nRandom Forest-------------------------------------------------------")
print("(1) train_test_split\n")
#(1) train_test_split

#train data와 test data분리
from sklearn.model_selection import train_test_split
#test_size=0.33 - train데이터(7)와 test data(3)를 7대3으로 나눈다
#random_state=42 data를 섞는다
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.25, random_state=42) #보통 42로 고정

from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(random_state=0)
rf.fit(X_train,y_train)
#테스트셋을 가지고 성능 확인
y_predict=rf.predict(X_test)

#성능 측정
def classification_performance_eval(y,y_predict):
    
    #A
    tp, tn, fp, fn = 0, 0, 0, 0

    for y, yp in zip(y_test,y_predict) :
        if (y == 'A' and yp == 'A'):
            tp += 1
        elif((y == 'A' and yp == 'B')or(y == 'A' and yp == 'C')):
            fn += 1 
        elif((y == 'B' and yp == 'B')or(y == 'C' and yp == 'C')) : 
            tn += 1
        else:
            fp += 1      
    #print("tp : ",tp," fn : ",fn," fp : ",fp," tn : ",tn)
    precision_a = (tp)/(tp+fp)
    recall_a = (tp) / (tp+fn)
    f1_score_a = 2*precision_a*recall_a / (precision_a + recall_a)

    #B

    tp, tn, fp, fn = 0, 0, 0, 0

    for y, yp in zip(y_test,y_predict) :
        if (y == 'B' and yp == 'B'):
            tp += 1
        elif((y == 'B' and yp == 'A')or(y == 'B' and yp == 'C')):
            fn += 1
        elif((y == 'A' and yp == 'A')or(y == 'C' and yp == 'C')) : 
            tn += 1
        else:
            fp += 1         

    #print("tp : ",tp," fn : ",fn," fp : ",fp," tn : ",tn)
    precision_b = (tp)/(tp+fp)
    recall_b = (tp) / (tp+fn)
    f1_score_b = 2*precision_b*recall_b / (precision_b + recall_b)

    #C
    tp, tn, fp, fn = 0, 0, 0, 0

    for y, yp in zip(y_test,y_predict) :
        if (y == 'C' and yp == 'C'):
            tp += 1
        elif((y == 'C' and yp == 'A')or(y == 'C' and yp == 'B')):
            fn += 1
        elif((y == 'A' and yp == 'A')or(y == 'B' and yp == 'B')) : 
            tn += 1
        else:
            fp += 1
    #print("tp : ",tp," fn : ",fn," fp : ",fp," tn : ",tn)
    precision_c = (tp)/(tp+fp)
    recall_c = (tp) / (tp+fn)
    f1_score_c = 2*precision_c*recall_c / (precision_c + recall_c)

    accuracy = (tp+tn)/(tp+tn+fp+fn)
    
    return accuracy,precision_a,precision_b,precision_c,recall_a,recall_b,recall_c,f1_score_a,f1_score_b,f1_score_c

acc, prec_a, prec_b, prec_c, rec_a, rec_b, rec_c, f1_a, f1_b, f1_c = classification_performance_eval(y_test,y_predict)

print("\naccuracy=%f"%acc)

print("A")
print("precision=%f"%prec_a)
print("recall=%f"%rec_a)
print("f1_score=%f\n"%f1_a)

print("B")
print("precision=%f"%prec_b)
print("recall=%f"%rec_b)
print("f1_score=%f\n"%f1_b)

print("C")
print("precision=%f"%prec_c)
print("recall=%f"%rec_c)
print("f1_score=%f\n"%f1_c)

#(2)K-Flod cross validation
print("(2) K-Flod cross validation\n")
#정확한 성능 구하기
from sklearn.model_selection import KFold
accuracy= []
precision_a = []
precision_b = []
precision_c = []
recall_a = []
recall_b = []
recall_c = []
f1_score_a = []
f1_score_b = []
f1_score_c = []
#n_splits값을 너무 작게 train이 충분히 안된다.
kf = KFold(n_splits=3, random_state=42, shuffle=True)
for train_index,test_index in kf.split(X):
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
    # 모델 변경
    
    #훈련셋으로 RandomForest 훈련
    rf=RandomForestClassifier(random_state=0)
    rf.fit(X_train,y_train)
    #테스트셋을 가지고 성능 확인
    y_predict=rf.predict(X_test)
    
    acc, prec_a, prec_b, prec_c, rec_a, rec_b, rec_c, f1_a, f1_b, f1_c = classification_performance_eval(y_test,y_predict) 
    accuracy.append(acc)
    precision_a.append(prec_a)
    precision_b.append(prec_b)
    precision_c.append(prec_c)
    recall_a.append(rec_a)
    recall_b.append(rec_b)
    recall_c.append(rec_c)
    f1_score_a.append(f1_a)
    f1_score_b.append(f1_b)
    f1_score_c.append(f1_c)

import statistics
print("\naverage_accuracy = ",statistics.mean(accuracy))
print("A")
print("average_precision = ",statistics.mean(precision_a))
print("average_recall = ",statistics.mean(recall_a))
print("average_f1_score = ",statistics.mean(f1_score_a))
print("B")
print("average_precision = ",statistics.mean(precision_b))
print("average_recall = ",statistics.mean(recall_b))
print("average_f1_score = ",statistics.mean(f1_score_b))
print("C")
print("average_precision = ",statistics.mean(precision_c))
print("average_recall = ",statistics.mean(recall_c))
print("average_f1_score = ",statistics.mean(f1_score_c))


SVM--------------------------------------------------------------------
(1) train_test_split


accuracy=0.652174
A
precision=0.555556
recall=0.555556
f1_score=0.555556

B
precision=0.375000
recall=0.500000
f1_score=0.428571

C
precision=0.500000
recall=0.875000
f1_score=0.636364

(2) K-Flod cross validation


average_accuracy =  0.6413043478260869
A
average_precision =  0.4618055555555556
average_recall =  0.6404761904761905
average_f1_score =  0.523202614379085
B
average_precision =  0.46875
average_recall =  0.5520833333333334
average_f1_score =  0.49661654135338346
C
average_precision =  0.4725274725274725
average_recall =  0.7256944444444444
average_f1_score =  0.5695065803761455

LogisticRegression------------------------------------------------------
(1) train_test_split


accuracy=0.608696
A
precision=0.533333
recall=0.800000
f1_score=0.640000

B
precision=0.142857
recall=0.250000
f1_score=0.181818

C
precision=0.500000
recall=0.555556
f1_score=0.526316

(2) K-Flod cross validat

In [ ]:
#csv 파일 읽어서 sql파일에 넣기

import pymysql
import pandas as pd

def load_db_score_data():
    #xlfile = 'C:/Users\yoon2/datascience/db_score_3_labels.xlsx'
    xlfile = './datascience/db_score_3_labels.xlsx'
    db_score = pd.read_excel(xlfile)
    
    #db connection 생성
    conn = pymysql.connect(host = 'localhost',user = 'root',password = 'taeyoon!', db = 'datascienceassignment4')
    curs = conn.cursor(pymysql.cursors.DictCursor)

    #겹칠 경우
    drop_sql = """drop table if exists db_score"""
    curs.execute(drop_sql)
    conn.commit()
    
    #직접 데이터프레임으로부터 테이블을 만들고 입력 하는 모듈
    import sqlalchemy
    database_username = 'root'
    database_password = 'taeyoon!'
    database_ip = 'localhost'
    database_name = 'datascienceassignment4'
    database_connection = sqlalchemy.create_engine('mysql+pymysql://{0}:{1}@{2}/{3}'.format(database_username,database_password,database_ip,database_name))
    
    #db_score 테이블 생성 + 데이터 입력
    db_score.to_sql(con=database_connection, name='db_score',if_exists='replace')
    
load_db_score_data()

conn = pymysql.connect(host = 'localhost',user = 'root',password = 'taeyoon!', db = 'datascienceassignment4')
curs = conn.cursor(pymysql.cursors.DictCursor)

sql="select * from db_score"
curs.execute(sql)

#fetchone이 메모리요소에서 보면 바람직하나 데이터양이 적어 fetchall사용
data=curs.fetchall()

curs.close()
conn.close()

#딕셔너리 형태가 튜플 형태의 리스트로 바뀐다.
X=[(t['homework'],t['discussion'],t['midterm'])for t in data]

#tuple의 리스트가 numpy로 바뀐다.
import numpy as np
X=np.array(X)

y=[(t['grade']) for t in data]
y=np.array(y)

#----------------------------SVM--------------------------------------------------
print("SVM--------------------------------------------------------------------")
print("(1) train_test_split\n")
#(1) train_test_split

#train data와 test data분리
from sklearn.model_selection import train_test_split
#test_size=0.33 - train데이터(7)와 test data(3)를 7대3으로 나눈다
#random_state=42 data를 섞는다
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.25, random_state=42) #보통 42로 고정

'''
#특성 스케일 - 특성 간 값의 범위 줄이
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)
'''
#훈련셋으로 SVM 훈련
from sklearn.svm import SVC
svm_model=SVC(kernel='rbf',C=8,gamma=0.1)
svm_model.fit(X_train,y_train)
#테스트셋을 가지고 성능 확인
y_predict = svm_model.predict(X_test) # 테스트
 

#성능 측정
def classification_performance_eval(y,y_predict):
    
    #A
    tp, tn, fp, fn = 0, 0, 0, 0

    for y, yp in zip(y_test,y_predict) :
        if (y == 'A' and yp == 'A'):
            tp += 1
        elif((y == 'A' and yp == 'B')or(y == 'A' and yp == 'C')):
            fn += 1 
        elif((y == 'B' and yp == 'B')or(y == 'C' and yp == 'C')) : 
            tn += 1
        else:
            fp += 1      
    #print("tp : ",tp," fn : ",fn," fp : ",fp," tn : ",tn)
    precision_a = (tp)/(tp+fp)
    recall_a = (tp) / (tp+fn)
    f1_score_a = 2*precision_a*recall_a / (precision_a + recall_a)

    #B

    tp, tn, fp, fn = 0, 0, 0, 0

    for y, yp in zip(y_test,y_predict) :
        if (y == 'B' and yp == 'B'):
            tp += 1
        elif((y == 'B' and yp == 'A')or(y == 'B' and yp == 'C')):
            fn += 1
        elif((y == 'A' and yp == 'A')or(y == 'C' and yp == 'C')) : 
            tn += 1
        else:
            fp += 1         

    #print("tp : ",tp," fn : ",fn," fp : ",fp," tn : ",tn)
    precision_b = (tp)/(tp+fp)
    recall_b = (tp) / (tp+fn)
    f1_score_b = 2*precision_b*recall_b / (precision_b + recall_b)

    #C
    tp, tn, fp, fn = 0, 0, 0, 0

    for y, yp in zip(y_test,y_predict) :
        if (y == 'C' and yp == 'C'):
            tp += 1
        elif((y == 'C' and yp == 'A')or(y == 'C' and yp == 'B')):
            fn += 1
        elif((y == 'A' and yp == 'A')or(y == 'B' and yp == 'B')) : 
            tn += 1
        else:
            fp += 1
    #print("tp : ",tp," fn : ",fn," fp : ",fp," tn : ",tn)
    precision_c = (tp)/(tp+fp)
    recall_c = (tp) / (tp+fn)
    f1_score_c = 2*precision_c*recall_c / (precision_c + recall_c)

    accuracy = (tp+tn)/(tp+tn+fp+fn)
    
    return accuracy,precision_a,precision_b,precision_c,recall_a,recall_b,recall_c,f1_score_a,f1_score_b,f1_score_c

acc, prec_a, prec_b, prec_c, rec_a, rec_b, rec_c, f1_a, f1_b, f1_c = classification_performance_eval(y_test,y_predict)

print("\naccuracy=%f"%acc)

print("A")
print("precision=%f"%prec_a)
print("recall=%f"%rec_a)
print("f1_score=%f\n"%f1_a)

print("B")
print("precision=%f"%prec_b)
print("recall=%f"%rec_b)
print("f1_score=%f\n"%f1_b)

print("C")
print("precision=%f"%prec_c)
print("recall=%f"%rec_c)
print("f1_score=%f\n"%f1_c)

#(2)K-Flod cross validation
print("(2) K-Flod cross validation\n")
#정확한 성능 구하기
from sklearn.model_selection import KFold
accuracy= []
precision_a = []
precision_b = []
precision_c = []
recall_a = []
recall_b = []
recall_c = []
f1_score_a = []
f1_score_b = []
f1_score_c = []
#n_splits값을 너무 작게 train이 충분히 안된다.
kf = KFold(n_splits=4, random_state=42, shuffle=True)

for train_index,test_index in kf.split(X):
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
    # 모델 변경

    #훈련셋으로 SVM 훈련
    svm_model=SVC(kernel='rbf',C=8,gamma=0.1)
    svm_model.fit(X_train,y_train)
    #테스트셋을 가지고 성능 확인
    y_predict = svm_model.predict(X_test) # 테스트
    acc, prec_a, prec_b, prec_c, rec_a, rec_b, rec_c, f1_a, f1_b, f1_c= classification_performance_eval(y_test,y_predict) 
    accuracy.append(acc)
    precision_a.append(prec_a)
    precision_b.append(prec_b)
    precision_c.append(prec_c)
    recall_a.append(rec_a)
    recall_b.append(rec_b)
    recall_c.append(rec_c)
    f1_score_a.append(f1_a)
    f1_score_b.append(f1_b)
    f1_score_c.append(f1_c)

import statistics
print("\naverage_accuracy = ",statistics.mean(accuracy))
print("A")
print("average_precision = ",statistics.mean(precision_a))
print("average_recall = ",statistics.mean(recall_a))
print("average_f1_score = ",statistics.mean(f1_score_a))
print("B")
print("average_precision = ",statistics.mean(precision_b))
print("average_recall = ",statistics.mean(recall_b))
print("average_f1_score = ",statistics.mean(f1_score_b))
print("C")
print("average_precision = ",statistics.mean(precision_c))
print("average_recall = ",statistics.mean(recall_c))
print("average_f1_score = ",statistics.mean(f1_score_c))

#----------------------------LogisticRegression------------------------------
print("\nLogisticRegression------------------------------------------------------")
print("(1) train_test_split\n")
#(1) train_test_split

#train data와 test data분리
from sklearn.model_selection import train_test_split
#test_size=0.33 - train데이터(7)와 test data(3)를 7대3으로 나눈다
#random_state=42 data를 섞는다
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.25, random_state=43) #보통 42로 고정


from sklearn.linear_model import LogisticRegression
ml=LogisticRegression(C=1000.0,random_state=0,max_iter=10000)
ml.fit(X_train,y_train)
#테스트셋을 가지고 성능 확인
y_predict=ml.predict(X_test)

#성능 측정
def classification_performance_eval(y,y_predict):
    
    #A
    tp, tn, fp, fn = 0, 0, 0, 0

    for y, yp in zip(y_test,y_predict) :
        if (y == 'A' and yp == 'A'):
            tp += 1
        elif((y == 'A' and yp == 'B')or(y == 'A' and yp == 'C')):
            fn += 1 
        elif((y == 'B' and yp == 'B')or(y == 'C' and yp == 'C')) : 
            tn += 1
        else:
            fp += 1      
    #print("tp : ",tp," fn : ",fn," fp : ",fp," tn : ",tn)
    precision_a = (tp)/(tp+fp)
    recall_a = (tp) / (tp+fn)
    f1_score_a = 2*precision_a*recall_a / (precision_a + recall_a)

    #B

    tp, tn, fp, fn = 0, 0, 0, 0

    for y, yp in zip(y_test,y_predict) :
        if (y == 'B' and yp == 'B'):
            tp += 1
        elif((y == 'B' and yp == 'A')or(y == 'B' and yp == 'C')):
            fn += 1
        elif((y == 'A' and yp == 'A')or(y == 'C' and yp == 'C')) : 
            tn += 1
        else:
            fp += 1         

    #print("tp : ",tp," fn : ",fn," fp : ",fp," tn : ",tn)
    precision_b = (tp)/(tp+fp)
    recall_b = (tp) / (tp+fn)
    f1_score_b = 2*precision_b*recall_b / (precision_b + recall_b)

    #C
    tp, tn, fp, fn = 0, 0, 0, 0

    for y, yp in zip(y_test,y_predict) :
        if (y == 'C' and yp == 'C'):
            tp += 1
        elif((y == 'C' and yp == 'A')or(y == 'C' and yp == 'B')):
            fn += 1
        elif((y == 'A' and yp == 'A')or(y == 'B' and yp == 'B')) : 
            tn += 1
        else:
            fp += 1
    #print("tp : ",tp," fn : ",fn," fp : ",fp," tn : ",tn)
    precision_c = (tp)/(tp+fp)
    recall_c = (tp) / (tp+fn)
    f1_score_c = 2*precision_c*recall_c / (precision_c + recall_c)

    accuracy = (tp+tn)/(tp+tn+fp+fn)
    
    return accuracy,precision_a,precision_b,precision_c,recall_a,recall_b,recall_c,f1_score_a,f1_score_b,f1_score_c

acc, prec_a, prec_b, prec_c, rec_a, rec_b, rec_c, f1_a, f1_b, f1_c = classification_performance_eval(y_test,y_predict)

print("\naccuracy=%f"%acc)

print("A")
print("precision=%f"%prec_a)
print("recall=%f"%rec_a)
print("f1_score=%f\n"%f1_a)

print("B")
print("precision=%f"%prec_b)
print("recall=%f"%rec_b)
print("f1_score=%f\n"%f1_b)

print("C")
print("precision=%f"%prec_c)
print("recall=%f"%rec_c)
print("f1_score=%f\n"%f1_c)


#(2)K-Flod cross validation
print("(2) K-Flod cross validation\n")
#정확한 성능 구하기
from sklearn.model_selection import KFold
accuracy= []
precision_a = []
precision_b = []
precision_c = []
recall_a = []
recall_b = []
recall_c = []
f1_score_a = []
f1_score_b = []
f1_score_c = []
#n_splits값을 너무 작게 train이 충분히 안된다.
kf = KFold(n_splits=3, random_state=500, shuffle=True)
#random_state 42에서 46으로 조정
for train_index,test_index in kf.split(X):
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
    # 모델 변경
    #특성 스케일 - 특성 간 값의 범위 줄이
    from sklearn.preprocessing import StandardScaler
    sc=StandardScaler()
    sc.fit(X_train)
    X_train_std = sc.transform(X_train)
    X_test_std = sc.transform(X_test)
    
    #훈련셋으로 LogisticRegression 훈련
    ml=LogisticRegression(C=1000,solver='lbfgs',random_state=42)
    ml.fit(X_train_std,y_train)
    #테스트셋을 가지고 성능 확인
    y_predict=ml.predict(X_test_std)
    
    acc, prec_a, prec_b, prec_c, rec_a, rec_b, rec_c, f1_a, f1_b, f1_c = classification_performance_eval(y_test,y_predict) 
    accuracy.append(acc)
    precision_a.append(prec_a)
    precision_b.append(prec_b)
    precision_c.append(prec_c)
    recall_a.append(rec_a)
    recall_b.append(rec_b)
    recall_c.append(rec_c)
    f1_score_a.append(f1_a)
    f1_score_b.append(f1_b)
    f1_score_c.append(f1_c)

import statistics
print("\naverage_accuracy = ",statistics.mean(accuracy))
print("A")
print("average_precision = ",statistics.mean(precision_a))
print("average_recall = ",statistics.mean(recall_a))
print("average_f1_score = ",statistics.mean(f1_score_a))
print("B")
print("average_precision = ",statistics.mean(precision_b))
print("average_recall = ",statistics.mean(recall_b))
print("average_f1_score = ",statistics.mean(f1_score_b))
print("C")
print("average_precision = ",statistics.mean(precision_c))
print("average_recall = ",statistics.mean(recall_c))
print("average_f1_score = ",statistics.mean(f1_score_c))


#----------------------------Random Forest------------------------------
print("\nRandom Forest-------------------------------------------------------")
print("(1) train_test_split\n")
#(1) train_test_split

#train data와 test data분리
from sklearn.model_selection import train_test_split
#test_size=0.33 - train데이터(7)와 test data(3)를 7대3으로 나눈다
#random_state=42 data를 섞는다
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.25, random_state=42) #보통 42로 고정

from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(random_state=0)
rf.fit(X_train,y_train)
#테스트셋을 가지고 성능 확인
y_predict=rf.predict(X_test)

#성능 측정
def classification_performance_eval(y,y_predict):
    
    #A
    tp, tn, fp, fn = 0, 0, 0, 0

    for y, yp in zip(y_test,y_predict) :
        if (y == 'A' and yp == 'A'):
            tp += 1
        elif((y == 'A' and yp == 'B')or(y == 'A' and yp == 'C')):
            fn += 1 
        elif((y == 'B' and yp == 'B')or(y == 'C' and yp == 'C')) : 
            tn += 1
        else:
            fp += 1      
    #print("tp : ",tp," fn : ",fn," fp : ",fp," tn : ",tn)
    precision_a = (tp)/(tp+fp)
    recall_a = (tp) / (tp+fn)
    f1_score_a = 2*precision_a*recall_a / (precision_a + recall_a)

    #B

    tp, tn, fp, fn = 0, 0, 0, 0

    for y, yp in zip(y_test,y_predict) :
        if (y == 'B' and yp == 'B'):
            tp += 1
        elif((y == 'B' and yp == 'A')or(y == 'B' and yp == 'C')):
            fn += 1
        elif((y == 'A' and yp == 'A')or(y == 'C' and yp == 'C')) : 
            tn += 1
        else:
            fp += 1         

    #print("tp : ",tp," fn : ",fn," fp : ",fp," tn : ",tn)
    precision_b = (tp)/(tp+fp)
    recall_b = (tp) / (tp+fn)
    f1_score_b = 2*precision_b*recall_b / (precision_b + recall_b)

    #C
    tp, tn, fp, fn = 0, 0, 0, 0

    for y, yp in zip(y_test,y_predict) :
        if (y == 'C' and yp == 'C'):
            tp += 1
        elif((y == 'C' and yp == 'A')or(y == 'C' and yp == 'B')):
            fn += 1
        elif((y == 'A' and yp == 'A')or(y == 'B' and yp == 'B')) : 
            tn += 1
        else:
            fp += 1
    #print("tp : ",tp," fn : ",fn," fp : ",fp," tn : ",tn)
    precision_c = (tp)/(tp+fp)
    recall_c = (tp) / (tp+fn)
    f1_score_c = 2*precision_c*recall_c / (precision_c + recall_c)

    accuracy = (tp+tn)/(tp+tn+fp+fn)
    
    return accuracy,precision_a,precision_b,precision_c,recall_a,recall_b,recall_c,f1_score_a,f1_score_b,f1_score_c

acc, prec_a, prec_b, prec_c, rec_a, rec_b, rec_c, f1_a, f1_b, f1_c = classification_performance_eval(y_test,y_predict)

print("\naccuracy=%f"%acc)

print("A")
print("precision=%f"%prec_a)
print("recall=%f"%rec_a)
print("f1_score=%f\n"%f1_a)

print("B")
print("precision=%f"%prec_b)
print("recall=%f"%rec_b)
print("f1_score=%f\n"%f1_b)

print("C")
print("precision=%f"%prec_c)
print("recall=%f"%rec_c)
print("f1_score=%f\n"%f1_c)

#(2)K-Flod cross validation
print("(2) K-Flod cross validation\n")
#정확한 성능 구하기
from sklearn.model_selection import KFold
accuracy= []
precision_a = []
precision_b = []
precision_c = []
recall_a = []
recall_b = []
recall_c = []
f1_score_a = []
f1_score_b = []
f1_score_c = []
#n_splits값을 너무 작게 train이 충분히 안된다.
kf = KFold(n_splits=3, random_state=42, shuffle=True)
for train_index,test_index in kf.split(X):
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
    # 모델 변경
    
    #훈련셋으로 RandomForest 훈련
    rf=RandomForestClassifier(random_state=0)
    rf.fit(X_train,y_train)
    #테스트셋을 가지고 성능 확인
    y_predict=rf.predict(X_test)
    
    acc, prec_a, prec_b, prec_c, rec_a, rec_b, rec_c, f1_a, f1_b, f1_c = classification_performance_eval(y_test,y_predict) 
    accuracy.append(acc)
    precision_a.append(prec_a)
    precision_b.append(prec_b)
    precision_c.append(prec_c)
    recall_a.append(rec_a)
    recall_b.append(rec_b)
    recall_c.append(rec_c)
    f1_score_a.append(f1_a)
    f1_score_b.append(f1_b)
    f1_score_c.append(f1_c)

import statistics
print("\naverage_accuracy = ",statistics.mean(accuracy))
print("A")
print("average_precision = ",statistics.mean(precision_a))
print("average_recall = ",statistics.mean(recall_a))
print("average_f1_score = ",statistics.mean(f1_score_a))
print("B")
print("average_precision = ",statistics.mean(precision_b))
print("average_recall = ",statistics.mean(recall_b))
print("average_f1_score = ",statistics.mean(f1_score_b))
print("C")
print("average_precision = ",statistics.mean(precision_c))
print("average_recall = ",statistics.mean(recall_c))
print("average_f1_score = ",statistics.mean(f1_score_c))


In [2]:
#csv 파일 읽어서 sql파일에 넣기

import pymysql
import pandas as pd

def load_db_score_data():
    xlfile = './datascience/db_score_3_labels.xlsx'
    db_score = pd.read_excel(xlfile)
    
    #db connection 생성
    conn = pymysql.connect(host = 'localhost',user = 'root',password = 'taeyoon!', db = 'datascienceassignment4')
    curs = conn.cursor(pymysql.cursors.DictCursor)

    #겹칠 경우
    drop_sql = """drop table if exists db_score"""
    curs.execute(drop_sql)
    conn.commit()
    
    #직접 데이터프레임으로부터 테이블을 만들고 입력 하는 모듈
    import sqlalchemy
    database_username = 'root'
    database_password = 'taeyoon!'
    database_ip = 'localhost'
    database_name = 'datascienceassignment4'
    database_connection = sqlalchemy.create_engine('mysql+pymysql://{0}:{1}@{2}/{3}'.format(database_username,database_password,database_ip,database_name))
    
    #iris 테이블 생성 + 데이터 입력
    db_score.to_sql(con=database_connection, name='db_score',if_exists='replace')
load_db_score_data()

In [9]:
import pymysql
import numpy as np

    conn = pymysql.connect(host = 'localhost',user = 'root',password = 'taeyoon!', db = 'datascienceassignment4')
    curs = conn.cursor(pymysql.cursors.DictCursor)

    sql="select * from db_score"
    curs.execute(sql)

    #fetchone이 메모리요소에서 보면 바람직하나 데이터양이 적어 fetchall사용
    data=curs.fetchall()

    #불러오는법
    #print(data)

    curs.close()
    conn.close()

In [12]:
#딕셔너리 형태가 튜플 형태의 리스트로 바뀐다.
X=[(t['sno'],t['homework'],t['discussion'],t['midterm'])for t in data]
print(X)


#print(type(X)) - <class 'list'>
#print(type(X[0])) - <class 'tuple'>

#tuple의 리스트가 numpy로 바뀐다.
X=np.array(X)
#print(X)

#print(X.shape) - (150, 4)

[(1, 19.37, 0, 35.0), (2, 20.0, 0, 34.12), (3, 19.37, 0, 34.3), (4, 18.12, 2, 31.5), (5, 20.0, 2, 33.25), (6, 19.37, 0, 24.5), (7, 19.37, 2, 23.8), (8, 16.87, 0, 28.87), (9, 19.37, 0, 29.75), (10, 20.0, 0, 31.5), (11, 19.37, 0, 29.57), (12, 19.37, 0, 31.5), (13, 19.37, 0, 29.75), (14, 20.0, 0, 31.5), (15, 18.12, 0, 28.0), (16, 19.37, 0, 24.5), (17, 20.0, 0, 30.62), (18, 19.37, 0, 30.62), (19, 19.37, 0, 24.5), (20, 19.37, 0, 28.0), (21, 19.37, 0, 35.0), (22, 16.87, 0, 35.0), (23, 19.37, 0, 29.75), (24, 19.37, 0, 28.0), (25, 19.37, 2, 35.0), (26, 16.87, 0, 30.45), (27, 19.37, 2, 22.75), (28, 19.37, 0, 35.0), (29, 16.25, 0, 26.25), (30, 16.87, 0, 28.0), (31, 15.0, 0, 28.87), (32, 20.0, 0, 30.62), (33, 19.37, 0, 23.45), (34, 20.0, 2, 26.25), (35, 18.12, 0, 28.0), (36, 20.0, 0, 28.0), (37, 16.87, 0, 24.5), (38, 20.0, 0, 25.37), (39, 13.75, 0, 33.25), (40, 20.0, 2, 24.5), (41, 19.37, 0, 24.5), (42, 19.37, 0, 22.75), (43, 19.37, 0, 22.57), (44, 17.5, 0, 29.75), (45, 19.37, 0, 30.62), (46, 19.

In [14]:
y=[1 if (t['grade']=='B') else -1 for t in data]
y=np.array(y)
print(y)
#print(y.shape) - (150,)

[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]


In [18]:
#train data와 test data분리

from sklearn.model_selection import train_test_split
#test_size=0.33 - train데이터(7)와 test data(3)를 7대3으로 나눈다
#random_state=42 data를 섞는다
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.33, random_state=42) #보통 42로 고정

#print(X_train)
#print(y_train)


from sklearn import tree
#모델 생성
dtree = tree.DecisionTreeClassifier()
dtree_model = dtree.fit(X_train, y_train) 

#예측 값
y_predict = dtree_model.predict(X_test)
print(y_predict)

[ 1 -1  1 -1 -1 -1  1 -1 -1  1 -1  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1
  1 -1 -1 -1  1 -1 -1]


In [19]:
#성능 측정
def classification_performance_eval(y,y_predict):
    tp, tn, fp, fn = 0, 0, 0, 0
    
    for y, yp in zip(y,y_predict) :
            if (y == 1 and yp == 1):
                tp += 1
            elif(y == 1 and yp == -1):
                fn += 1
            elif(y== -1 and yp == 1) : 
                fp += 1
            else:
                tn += 1
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    precision = (tp)/(tp+fp)
    recall = (tp) / (tp+fn)
    f1_score = 2*precision*recall / (precision + recall)
    
    return accuracy,precision,recall,f1_score

acc, prec, rec, f1 = classification_performance_eval(y_test,y_predict)

print("accuracy=%f"%acc)
print("precision=%f"%prec)
print("recall=%f"%rec)
print("f1_score=%f"%f1)

accuracy=0.967742
precision=1.000000
recall=0.888889
f1_score=0.941176


In [49]:
#정확한 성능 구하기
from sklearn.model_selection import KFold

accuracy= []
precision = []
recall = []
f1_score = []

#n_splits값을 너무 작게 train이 충분히 안된다.
kf = KFold(n_splits=5, random_state=42, shuffle=True)

for train_index,test_index in kf.split(X):
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
    
    dtree = tree.DecisionTreeClassifier()
    dtree = dtree_model.fit(X_train,y_train)
    y_predict = dtree_model.predict(X_test)
    acc, prec, rec, f1 = classification_performance_eval(y_test,y_predict) 
    accuracy.append(acc)
    precision.append(prec)
    recall.append(rec)
    f1_score.append(f1)
    
#print(accuracy)

In [50]:
import statistics

print("average_accuracy = ",statistics.mean(accuracy))
print("average_precision = ",statistics.mean(precision))
print("average_recall = ",statistics.mean(recall))
print("average_f1_score = ",statistics.mean(f1_score))

average_accuracy =  0.967251461988304
average_precision =  0.9818181818181818
average_recall =  0.9314285714285714
average_f1_score =  0.9528693528693528
